In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets

## Data

In [2]:
mnist  = datasets.mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
def mould(data):
  return data.reshape(-1, 28,28,1).astype("float32") / 255.0

x_train, x_test = mould(x_train), mould(x_test)

## Model


In [21]:
class CNNBlock(layers.Layer):
  def __init__(self, out_channels, kernel_size=3):
    super().__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()

  def call(self, x, training = False):
    out = self.conv(x)
    out = self.bn(out)
    out = tf.nn.relu(out)
    return out

In [24]:
class ResBlock(layers.Layer):
  def __init__(self, channels):
    super().__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(channels[2])
    self.pool = layers.MaxPooling2D()
    self.identity_mapping = layers.Conv2D(channels[1], 1)
  
  def call(self, x, training=False):
    out = self.cnn1(x, training=training)
    out = self.cnn2(out, training=training)
    out = self.cnn3(
        out + self.identity_mapping(x), training = training
    )
    out = self.pool(out)
    return out

In [26]:
class ResNet_Lite(keras.Model):
  def __init__(self, num_classes):
    super().__init__()
    self.block1 = ResBlock([32,32,64])
    self.block2 = ResBlock([64, 128, 128])
    self.block3 = ResBlock([128, 256, 512])
    self.pool = layers.GlobalAveragePooling2D()
    self.mlp = layers.Dense(num_classes)
  
  def call(self, x, training=False):

    out = self.block1(x, training)
    out = self.block2(out, training)
    out = self.block3(out, training)
    out = self.pool(out)

    return self.mlp(out)




In [25]:
model_1 = keras.Sequential(
    [
     keras.Input(shape=(28,28,1)),
     CNNBlock(32),
     CNNBlock(64),
     CNNBlock(128),
     layers.Flatten(),
     layers.Dense(10),
    ]
)

model_1.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_block_6 (CNNBlock)      (None, 26, 26, 32)        448       
                                                                 
 cnn_block_7 (CNNBlock)      (None, 24, 24, 64)        18752     
                                                                 
 cnn_block_8 (CNNBlock)      (None, 22, 22, 128)       74368     
                                                                 
 flatten_2 (Flatten)         (None, 61952)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                619530    
                                                                 
Total params: 713,098
Trainable params: 712,650
Non-trainable params: 448
_________________________________________________________________


In [13]:
model.fit(x_train, y_train, batch_size = 64, epochs=10, verbose = 2)

Epoch 1/10
938/938 - 20s - loss: 0.4659 - accuracy: 0.9477 - 20s/epoch - 21ms/step
Epoch 2/10
938/938 - 8s - loss: 0.0880 - accuracy: 0.9811 - 8s/epoch - 8ms/step
Epoch 3/10
938/938 - 8s - loss: 0.0392 - accuracy: 0.9884 - 8s/epoch - 9ms/step
Epoch 4/10
938/938 - 8s - loss: 0.0279 - accuracy: 0.9915 - 8s/epoch - 8ms/step
Epoch 5/10
938/938 - 8s - loss: 0.0230 - accuracy: 0.9924 - 8s/epoch - 8ms/step
Epoch 6/10
938/938 - 8s - loss: 0.0229 - accuracy: 0.9926 - 8s/epoch - 9ms/step
Epoch 7/10
938/938 - 8s - loss: 0.0176 - accuracy: 0.9947 - 8s/epoch - 8ms/step
Epoch 8/10
938/938 - 8s - loss: 0.0152 - accuracy: 0.9948 - 8s/epoch - 9ms/step
Epoch 9/10
938/938 - 8s - loss: 0.0136 - accuracy: 0.9953 - 8s/epoch - 9ms/step
Epoch 10/10
938/938 - 8s - loss: 0.0107 - accuracy: 0.9965 - 8s/epoch - 9ms/step


In [14]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 1s - loss: 0.0453 - accuracy: 0.9876 - 928ms/epoch - 6ms/step


[0.045340634882450104, 0.9876000285148621]

In [32]:
model = keras.Sequential(
    [
    keras.Input(shape=(28,28,1)),
    ResNet_Lite(10)
    ]
)

In [33]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_net__lite_3 (ResNet_Lit  (None, 10)               1961258   
 e)                                                              
                                                                 
Total params: 1,961,258
Trainable params: 1,958,570
Non-trainable params: 2,688
_________________________________________________________________


In [35]:
# from tensorflow.python.ops.gen_nn_ops import LRN
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [36]:
model.fit(x_train, y_train, batch_size = 64, epochs = 10, verbose = 2)

Epoch 1/10
938/938 - 22s - loss: 0.0818 - accuracy: 0.9743 - 22s/epoch - 23ms/step
Epoch 2/10
938/938 - 17s - loss: 0.0338 - accuracy: 0.9895 - 17s/epoch - 19ms/step
Epoch 3/10
938/938 - 18s - loss: 0.0280 - accuracy: 0.9917 - 18s/epoch - 19ms/step
Epoch 4/10
938/938 - 18s - loss: 0.0241 - accuracy: 0.9926 - 18s/epoch - 19ms/step
Epoch 5/10
938/938 - 18s - loss: 0.0198 - accuracy: 0.9937 - 18s/epoch - 19ms/step
Epoch 6/10
938/938 - 18s - loss: 0.0194 - accuracy: 0.9937 - 18s/epoch - 19ms/step
Epoch 7/10
938/938 - 18s - loss: 0.0151 - accuracy: 0.9950 - 18s/epoch - 19ms/step
Epoch 8/10
938/938 - 18s - loss: 0.0148 - accuracy: 0.9950 - 18s/epoch - 19ms/step
Epoch 9/10
938/938 - 18s - loss: 0.0140 - accuracy: 0.9956 - 18s/epoch - 19ms/step
Epoch 10/10
938/938 - 18s - loss: 0.0108 - accuracy: 0.9966 - 18s/epoch - 19ms/step


In [37]:
model.evaluate(x_test, y_test, batch_size = 64, verbose = 2)

157/157 - 1s - loss: 0.0246 - accuracy: 0.9924 - 1s/epoch - 10ms/step


[0.02463269606232643, 0.9923999905586243]